In [ ]:
import numpy as np
import pandas as pd
import time
import psutil
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train=pd.read_csv('X_train3.csv')
X_train=X_train.to_numpy()
X_test=pd.read_csv('X_test3.csv')
X_test=X_test.to_numpy()
y_train=pd.read_csv('y_train3.csv')
y_train=y_train.to_numpy()
y_train=y_train.reshape(-1)
y_test=pd.read_csv('y_test3.csv')
y_test=y_test.to_numpy().reshape(-1)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

In [ ]:
import os
from threading import Thread

In [ ]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [ ]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 5

In [ ]:
time.sleep(3)

In [ ]:
weights = np.zeros(X_train.shape[1])
bias = 0
lr = 0.01
# C = 0.1 # change C=10 or 1 if high attributes are involved

In [ ]:
results={}
def svm_sgd(X, y, learning_rate=0.01, epochs=3):
    weights = np.zeros(X.shape[1])
    bias = 0
    lr = 0.01
    C = 1
    train_time_start=time.time()
    for _ in range(3):
      for i in range(X_train.shape[0]):
        xi=X_train[i]
        yi=y_train[i]

        mar=1-yi*(np.dot(xi,weights)+bias)

        if mar>0:
          weights=weights-lr*(weights-C*yi*xi)
          bias=bias-lr*(-C*yi)
        else:
          weights=weights-lr*weights
    results['weights']=weights
    results['bias']=bias
    train_time_end=time.time()
    results['train_time']=train_time_end-train_time_start
    print(f"Training time: {results['train_time']} seconds")

In [ ]:
# function to train the model

train_thread = Thread(target=svm_sgd, args=(X_train, y_train))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

Training time: 0.10576987266540527 seconds
CPU utilization: 5.0% | RAM utilization: 1.5620527807163935%
CPU utilization: 1.0% | RAM utilization: 1.5620527807163935%
CPU utilization: 1.0% | RAM utilization: 1.5620527807163935%
CPU utilization: 0.0% | RAM utilization: 1.5620527807163935%
CPU utilization: 0.0% | RAM utilization: 1.5620527807163935%
Total CPU utilization over 5 seconds: 7.0%
CPU usage per second: [5.0, 1.0, 1.0, 0.0, 0.0]
RAM usage per second: [1.5620527807163935, 1.5620527807163935, 1.5620527807163935, 1.5620527807163935, 1.5620527807163935]
Total CPU resource consumption: 0.07 CPU-seconds


In [ ]:
print("Weights from SGD:", results['weights'])
print("Bias from SGD:", results['bias'])

Weights from SGD: [ 0.03536439  0.03037078 -0.01829586  0.00793404 -0.00979498 -0.00476678
 -0.01825768 -0.00291534  0.04729051  0.03380606  0.03417382  0.01134454
  0.00624315 -0.01069552  0.00171848]
Bias from SGD: -1.1000000000000008


In [ ]:
def predict(X, w, b):
    return np.sign(np.dot(X, w) + b)

In [ ]:
test_time_start=time.time()
y_pred=predict(X_test,results['weights'],results['bias'])
test_time_end=time.time()
results['test_time']=test_time_end-test_time_start
print(f"Testing time: {results['test_time']} seconds")

Testing time: 0.001636505126953125 seconds


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8481481481481481

In [ ]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8481481481481481

In [ ]:
import joblib
joblib.dump((results['weights'], results['bias'], accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_manual_sgd.pkl')

['variables_manual_sgd.pkl']

In [ ]:
y_pred

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [ ]:
y_test

array([-1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1,
       -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1,  1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1